<a href="https://colab.research.google.com/github/bharadwaj103/Aiml-Lab/blob/main/AIML_Lab8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>